# Metaphor algorithm

1. Parse file and list objects & mapto implied by subsets, common structures/covers
2. Match structures  
    a. Match types (how to deal with mappable ones? eg. segment->linear)  
    b. Match affected and covers
3. Match representations (keeping the structure consistent)



In [158]:
import yaml
import re
import enum
from pprint import pprint

def get_spec(file_path):
  with open(file_path, 'r') as file_handle:
    spec = yaml.safe_load(file_handle)
  return spec

In [96]:
ex_spec = get_spec('video-editor.yaml')

In [97]:
def process_listable(target, func):
  if target is None:
    return

  if type(target) == list:
    for target_item in target:
      func(target_item)
  else:
    func(target)

## Parsing for objects

In [151]:
def strip_type(obj):
  return re.sub(r'\((\w|-)*\) ', '', obj)

def register_object(registry, obj):
  # -> means mapto
  # . means subset
  # / means component  # TODO: actually do this!!
  assert(type(obj) is str)
  obj = strip_type(obj)

  arrow_array = obj.split('->')
  for arrow_idx, arrow_term in enumerate(arrow_array):
    
    dot_array = arrow_term.split('.')
    for dot_idx, dot_term in enumerate(dot_array):
      subject = '.'.join(dot_array[:dot_idx + 1]) # join up to idx
      if not registry.get(subject):
        registry[subject] = set()
      
      if dot_idx > 0:
        # every sequence maps to the previous element eg. a.b.c => {a.b.c: {a.b}, a.b: {a}}
        previous = '.'.join(dot_array[:dot_idx])
        registry[subject].add(previous)
    
    if arrow_idx > 0:
      # a pair of arrows lhs->rhs => {lhs: {rhs}}
      lhs = registry[arrow_array[arrow_idx - 1]]
      rhs = arrow_array[arrow_idx]
      lhs.add(rhs)

In [99]:
# Tests
print('testing: a')
test = {}
print('expected:')
pprint({'a': set()})
print('got:')
register_object(test, 'a')
pprint(test)

print()

print('testing: a.b.c')
test = {}
print('expected:')
pprint({'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}})
print('got:')
register_object(test, 'a.b.c')
pprint(test)

print()

print('testing: a->b->c')
test = {}
print('expected:')
pprint({'a': {'b'}, 'b': {'c'}, 'c': set()})
print('got:')
register_object(test, 'a->b->c')
pprint(test)

print()

print('testing: a.b->x->z.w')

test = {}
register_object(test, 'a.b->x->z.w')
print('expected:')
print({'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}})
print('got:')
pprint(test)


testing: a
expected:
{'a': set()}
got:
{'a': set()}

testing: a.b.c
expected:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}
got:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}

testing: a->b->c
expected:
{'a': {'b'}, 'b': {'c'}, 'c': set()}
got:
{'a': {'b'}, 'b': {'c'}, 'c': set()}

testing: a.b->x->z.w
expected:
{'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}
got:
{'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}


In [152]:
def register_struct_objects(struct, registry, process_func):
  if struct.get('type') == 'group':
    # Groups also behave as objects, so register them
    name = struct.get('name')
    if name is None:
      print('Warning! No name provided for group. Using `NO NAME PROVIDED` instead. TODO: generate ID.')
      name = 'NO NAME PROVIDED'

    if registry.get(name) is None:
      registry[name] = set()
    
    # TODO: do groups map to their elements? Really, does the transitive property apply? My hunch is no, but need to think more

  process_listable(struct.get('affects'), process_func)
  process_listable(struct.get('covered-by'), process_func)
  # TODO: relate all of the objects affected/covered by a structure. 

  for derivative in struct.get('structures', []):
    register_struct_objects(derivative, objects=registry, process_func=process_func)


def register_repr_objects(repr, registry, process_func):
  repr_type = repr.get('type', '')
  # print(repr_type)
  for repr_item in repr.get('objects', []):
    assert(len(repr_item.values()) == 1)
    repr_obj, target_objs = list(repr_item.items())[0]

    process_listable(target_objs, process_func)

    # Map every item to the associated representational object
    # eg. {message: {textbox}, author: {textbox}}
    def register_repr_maps(target):
      # note: this is a bit backwards compared to the syntax!
      if registry.get(target) is None:
        registry[target] = set()
      registry[target].add(repr_obj)
      # objects[target].add(repr_type + '/' + repr_obj)  # when we prefix the core stuff
    process_listable(target_objs, register_repr_maps)



def register_spec_objects(spec, registry={}): # {object: [mapto-targets]}
  # TODO: deal with `objects` block
  def register_object_here(target):
    register_object(registry, target)
  
  process_listable(spec.get('objects'), register_object_here)

  for struct in spec.get('structures', []):
    register_struct_objects(struct, registry, register_object_here)
    
  for repr in spec.get('representations', []):
    register_repr_objects(repr, registry, register_object_here)
  
  return registry

In [153]:
pprint(register_spec_objects(ex_spec))

{'playhead': {'vline', 'videos.in-editor/images'},
 'playhead->videos.in-editor/images': {'rect'},
 'timestamps': {'vline'},
 'tracks': {'region'},
 'videos': set(),
 'videos.in-editor': {'rect', 'videos'},
 'videos.in-editor/images': {'videos'},
 'videos/first-frame': {'regions'}}


In [154]:
core_spec = get_spec('core.yaml')
core_objs = {}

# TODO: probably need to consider prefixing gui on these
for repr_type in core_spec.get('representation-types', []):
  # print(repr_type['name'])
  register_spec_objects(repr_type, core_objs)

# register_spec_objects(core_spec, core_objs)

pprint(core_objs)

{'hlines': {'lines'},
 'icons': {'regions'},
 'lines': {'regions'},
 'points': set(),
 'rects': {'regions'},
 'regions': set(),
 'vlines': {'lines'}}


In [155]:
def get_node_join(left, right, relations, visited=set()):
  # TODO: I bet there's 
  assert(type(left) is str)
  assert(type(right) is str)

  # Trace print
  # print(f"{left:<8} {right:<}")

  if left in visited or right in visited:
    # To avoid cycles, abort if we've already visited a node
    # NOTE: Not like 100% sure this is clean
    print("WARNING! Cycle detected in the relations.")
    return None

  if left == right:
    return left
  
  for next_left in relations[left]:
    left_res = get_node_join(next_left, right, relations, visited=visited)
    if left_res is None:
      # When you reach the leaf of a spanning tree, pop back up to recurse
      for next_right in relations[right]:
        return get_node_join(left, next_right, relations, visited=visited)
    else:
      return left_res

  if len(relations[right]) == 0 and len(relations[left]) == 0:
    return None


In [156]:
print('vlines, hlines => ' + str(get_node_join('vlines', 'hlines', core_objs)))
print('rects, hlines => ' + str(get_node_join('rects', 'hlines', core_objs)))
print('points, hlines => ' + str(get_node_join('points', 'hlines', core_objs)))

vlines, hlines => lines
rects, hlines => regions
points, hlines => None


## Prepare structure relations
We want to be able to say that two structures have the same time if they both map to a common structure (if they have common structure eg. gui.stack) and structure type.

In [ ]:
def register_struct_relation(registry, )

## Find Overlaps
An overlap is a spec that is found in each side. There are usually a lot, and ideally we want the "maximal" overlaps (and not all of its the subsets).

The strategy is to find an overlap, exploit it as much as possible, then move on.

How do you know if you're done?  
-- You've looped through all things and stopped adding things.

When do you duplicate your candidate?  
-- I guess when you're going back up  

Are we going depth first or breadth first?  
-- Depth first makes the most sense I think  

How do you even recurse, since every node behaves differently?   
-- Well I guess you need a traversal function that takes a "deal with node" function with parameters for local data  

How do you traverse two specs at once?  
-- Probably like the relation meet algo: progress in one, and when it's done, pop back to the top before recursing

What if I look at _just_ the structures, then extend them with representations and behavior?  
-- That can work, as long as I can match representations on their own (ie. if no structure stuff maps)

How do I compare structures.affects without m*n valid matches?  
-- The greedy approach feels like reaching into the uses for those objects, and basically buidling the graph.
-- I think a smarter way to do it is to match all the structures/representations/behaviors based on type and used fields, then in a second pass flesh them out with objects. 

In [159]:
# Get object relation registry
def traverse_spec(spec, func):
  # Navigate Structures
  for struct in spec.get('structures', []):
    func(struct)
  # Hm.

class Context(enum.Enum):
  Structure = enum.auto()
  Representation = enum.auto()
  Behavior = enum.auto()

def add_to_overlap():
  pass

def compare_structs(sinister, dextera, struct_relations):
  # Check Type
  common_type = None
  if sinister.get('type') == dextera.get('type'):
    common_type = sinister.get('type')
  # TODO: find join under struct_relations
  
  # TODO: how do I compare these without m*n valid matches?
  # Find overlap on affects
  pass
  
  # Find overlap on overlaps
  pass

def compare_nodes(s_node, d_node, context, obj_relations, struct_relations):
  assert(type(context) is Context)

  if context == Context.Structure:
    compare_structs(s_node, d_node, struct_relations)
  elif context == Context.Representation:
    pass
  elif context == Context.Behavior:
    pass

def traverse_specs(sinister, dextera, node_func):
  pass